In [4]:
import pyro
import pyro.distributions as dist
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.infer.mcmc as mcmc
import torch.distributions


def bayesian_pca(data, latent_dim):
    # Define model parameters
    n, p = data.shape
    sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
    mu = torch.zeros(p)
    cov = sigma * torch.eye(p)

    # Mask the lower right corner of the data
    mask = torch.ones(n, p)
    mask[-20:, -2:] = 0
    masked_data = data * mask
    

    # Define the custom mask distribution
    def mask_dist(mean, covariance):
        masked_mean = mask * mean
        return dist.MultivariateNormal(masked_mean, covariance_matrix=covariance)
    
    # Define the latent variables for the masked data

    Z_mean = pyro.param("Z.mean", torch.zeros(n, latent_dim))
    Z_cov = pyro.param("Z.cov", torch.eye(latent_dim))
    Z = pyro.sample("Z", dist.MultivariateNormal(Z_mean, Z_cov))

    W_mean = pyro.param("W.mean", torch.zeros(latent_dim, p))
    W_cov = pyro.param("W.cov", torch.eye(p))
    W = pyro.sample("W", dist.MultivariateNormal(W_mean, W_cov))
    
    X = pyro.sample("X", mask_dist(Z @ W, cov * torch.eye(p)), obs=masked_data)

    # Return the estimated latent variables
    return X
    

In [10]:
data

tensor([[ 1397.5305,   464.8815,  -811.0114, -1125.1631,   827.1343],
        [  221.4790,    81.9377,   241.4629,   182.1595,  -764.8069],
        [ -246.9436,    96.4646,  -385.0945,  -644.1563,  -459.4985],
        [  466.0563,   310.5362,  -195.2516,  -365.9461,   -29.9426],
        [ -126.3790,   420.2422,  -968.7613,   115.9764,    73.2304],
        [  415.5353,   -32.5359,   -93.1402,  -571.9332,  -216.0164],
        [  277.3107,     3.2673,   672.4340,    95.9534,  -500.0756],
        [  -93.8707,  -180.2603,    71.5380,     2.6049,   267.7835],
        [ -131.2750,  -267.8922,  -963.8295, -1092.0708,   218.3588],
        [ -191.2837,  -594.8768,  -730.9603,  -345.2110,  -107.5722]])

In [6]:
# Define model

#Number of Dimensions post PCA, hyperparameter ?
latent_dim = 3
model = bayesian_pca

# Generate Data
n, p = 10, 5
data = torch.randn(n, p) * 500



# Run MCMC
num_samples = 100

#Lower warmup steps for code to run
warmup_steps = 200
kernel = mcmc.NUTS(model)
mcmc_run = mcmc.MCMC(kernel, num_samples=num_samples, warmup_steps=warmup_steps)
#Apply MCMC to our data.
mcmc_run.run(data, latent_dim)
pyro.clear_param_store()






Sample: 100%|██████████| 300/300 [05:46,  1.16s/it, step size=1.41e-04, acc. prob=0.345]


In [7]:
# Extract posterior samples, includes W, Z, and X. W is transformation, Z is weights, X is our data. 
#Sigma is our covariance matrix for X, assumed to be diagonal for PPCA I believe. 
posterior_samples = mcmc_run.get_samples()

# Extract W, sigma, and Z samples
W_samples = posterior_samples["W"]
sigma_samples = posterior_samples["sigma"]
Z_samples = posterior_samples["Z"]

print(W_samples.size())
print(Z_samples.size())


Z_samples

torch.Size([100, 3, 5])
torch.Size([100, 10, 3])


tensor([[[-29.6602,  14.1690,  19.4095],
         [  1.5899,  -0.4540,   7.9255],
         [ -1.5371,   7.6422,  -9.6393],
         ...,
         [  0.5550,  -6.0109,  -1.6433],
         [-17.2170,  -0.3782, -16.2096],
         [-16.1797, -12.7158, -15.6216]],

        [[-29.6684,  14.1698,  19.3950],
         [  1.5880,  -0.4531,   7.9247],
         [ -1.5383,   7.6260,  -9.6396],
         ...,
         [  0.5554,  -6.0048,  -1.6417],
         [-17.2175,  -0.3763, -16.2168],
         [-16.1787, -12.6978, -15.6251]],

        [[-29.6661,  14.1725,  19.3870],
         [  1.5898,  -0.4539,   7.9259],
         [ -1.5423,   7.6270,  -9.6385],
         ...,
         [  0.5542,  -6.0009,  -1.6414],
         [-17.2239,  -0.3680, -16.2221],
         [-16.1831, -12.6858, -15.6323]],

        ...,

        [[-28.8245,  13.8792,  20.3109],
         [  1.8854,  -0.6647,   8.2837],
         [ -1.8405,   7.7021,  -9.7676],
         ...,
         [  0.4406,  -5.8224,  -1.9566],
         [-17.8511,   

In [8]:
Z_samples[-1] @ W_samples[-1]

tensor([[1397.5576,  464.9338, -811.0127,  -66.5389,  -59.8559],
        [ 221.5226,   81.9686,  241.4845,    3.7030,    5.3524],
        [-246.9532,   96.4928, -385.0949,  -15.7925,  -16.7233],
        [ 466.0794,  310.5636, -195.2890,  -25.1117,  -21.8805],
        [-126.3983,  420.1721, -968.7817,  -52.4721,  -51.9528],
        [ 415.5706,  -32.4499,  -93.2189,   -7.5092,   -6.2844],
        [ 277.2784,    3.2286,  672.4957,   22.9628,   25.0001],
        [ -93.8814, -180.2111,   71.5173,   10.7733,    9.4946],
        [-131.2433, -267.8916, -963.8311,  -26.2018,  -29.6411],
        [-191.2900, -594.8683, -730.9649,   -3.9419,   -9.1393]])

In [9]:
data

tensor([[ 1397.5305,   464.8815,  -811.0114, -1125.1631,   827.1343],
        [  221.4790,    81.9377,   241.4629,   182.1595,  -764.8069],
        [ -246.9436,    96.4646,  -385.0945,  -644.1563,  -459.4985],
        [  466.0563,   310.5362,  -195.2516,  -365.9461,   -29.9426],
        [ -126.3790,   420.2422,  -968.7613,   115.9764,    73.2304],
        [  415.5353,   -32.5359,   -93.1402,  -571.9332,  -216.0164],
        [  277.3107,     3.2673,   672.4340,    95.9534,  -500.0756],
        [  -93.8707,  -180.2603,    71.5380,     2.6049,   267.7835],
        [ -131.2750,  -267.8922,  -963.8295, -1092.0708,   218.3588],
        [ -191.2837,  -594.8768,  -730.9603,  -345.2110,  -107.5722]])

In [176]:
reconstructed_X = Z_samples @ W_samples #Data_Rep Drawn from posterior Distribution

m = torch.distributions.MultivariateNormal(torch.zeros(100,5), torch.eye(5))
print(m.sample().size())


log_prob = []
test_prob_list = []
for w,z,sig,x in zip(W_samples, Z_samples, sigma_samples, reconstructed_X):
    #w is 2*5, z is 100*2, multiplied is 100*5. )
    #Calculate log probability that distribution 
    
    #Create new Y distribution based off our parameters
    sample_dist = dist.MultivariateNormal((Z_mean@W_mean)[-20:, -2:], covariance_matrix=(torch.eye(2)*sig))
    y_pred  = sample_dist.sample()
    
    #Calculate the likelihood given this sample
    y_pred_likelihood = sample_dist.log_prob(y_pred).sum()
    
    #print(y_pred_likelihood)
    
    #Now, we want to calculate the likelihood of the actual data. 
    data_test = data[-20:, -2:,]
    
    test_prob = dist.MultivariateNormal((Z_mean @ W_mean)[-20:, -2:], covariance_matrix=(torch.eye(2)*sig)).log_prob(data_test).sum()
    
    #print(test_prob)

    #break
    
    log_prob.append(y_pred_likelihood) 
    test_prob_list.append(test_prob)

count = sum([1 for x, y in zip(log_prob, test_prob_list) if x > y])
percent_likelihood = count / len(log_prob)
        
print("Percentage of Test distribution more likely than Y_Pred is" + str(percent_likelihood))
        

        
#Keep the same dimensionality, mask random values for train, then evaluate on full. 

#Hold out bottom corner, don't observe the outcomes. for the columns that were masked, create replicate timeseries for training, compare log probailities of replicated to log probabilities of the held out sequences. 

#Hold out 10 to 20 units, 2 time steps. Set up experimental design, randomly select units, last timesteps. 

torch.Size([100, 5])
Percentage of Test distribution more likely than Y_Pred is1.0


In [90]:
#Construct PPC from Posterior Sampled X, and New X from draws in Data. 


def PPC(data_new, data_obs): 
    
    #Y_Obs is observed data we feed into the model. 
    #Y_New is observed data we don't have in the model. 
    #Y_Rep is data we draw from the posterior distribution using Y_Obs. 
    
    
    




SyntaxError: unexpected EOF while parsing (<ipython-input-90-85ffe9619cf9>, line 13)

In [113]:
z = torch.randn(100, 2)

w = torch.randn(2, 5)

sig = 5

examplesamp = dist.MultivariateNormal((z@w), covariance_matrix=(torch.eye(100)*sig))

examplesamp.sample()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x5 and 100x100)

In [ ]:
data.size() # Original Data, 100 samples of 5 dimensions each 

In [ ]:
#Reconstruct PCA data by multiplying centered data with the average weight. 
X= W @ Z



W_mean = W_samples.mean(dim=0)
pca_data = torch.matmul(data - data.mean(dim=0), W_mean.T)